# 기본 베이스라인

미션 1: 베이스라인의 코드마다 간단하게 기능을 주셕으로 달아주세요.

(코드 흐름을 이해하기 위해서)

In [6]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# 랜덤 시드를 고정하여 실험의 재현성을 확보하는 함수
def seed_everything(seed):
    random.seed(seed)  # 파이썬 기본 random 모듈 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed)  # 파이썬 해시 시드 고정
    np.random.seed(seed)  # numpy 시드 고정

seed_everything(42)  # 시드를 42로 고정

# 학습(train) 데이터 및 테스트(test) 데이터를 불러옴
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

# 학습 데이터에서 'ID'와 'TARGET' 컬럼을 제외한 나머지 데이터를 독립 변수로 사용
x_train = train.drop(['ID', 'TARGET'], axis=1)
# 'TARGET' 컬럼은 종속 변수 (예측 대상)
y_train = train['TARGET']

# 테스트 데이터에서 'ID' 컬럼을 제외한 나머지 데이터를 예측에 사용할 독립 변수로 사용
x_test = test.drop('ID', axis=1)

# 범주형 데이터인 '요일'과 '범죄발생지' 컬럼에 라벨 인코딩(Label Encoding)을 적용
ordinal_features = ['요일', '범죄발생지']
for feature in ordinal_features:
    le = LabelEncoder()  # 라벨 인코딩 객체 생성
    le.fit(x_train[feature])  # 학습 데이터에 대해 라벨 인코딩 학습
    x_train[feature] = le.transform(x_train[feature])  # 학습 데이터에 인코딩 적용
    
    # 테스트 데이터에도 동일한 라벨 인코딩 적용
    for label in np.unique(x_test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)  # 새로운 값이 있으면 클래스 목록에 추가
    x_test[feature] = le.transform(x_test[feature])  # 테스트 데이터에 인코딩 적용

# 학습 데이터를 80%는 학습용, 20%는 검증용으로 분리 (train/validation split)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 의사결정나무 모델을 정의하고 학습
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)  # 학습 데이터로 모델 학습

# 검증 데이터에 대해 예측
y_pred = model.predict(x_val)

# 예측값과 실제값을 비교하여 F1 스코어(평균 F1)를 계산
f1 = f1_score(y_val, y_pred, average='macro')
print(f'F1 Score on the validation set: {f1:.4f}')

# 학습된 모델을 사용해 테스트 데이터에 대한 예측 수행
pred = model.predict(x_test)

# 제출 파일 생성: sample_submission.csv 파일을 읽어와 예측 결과를 TARGET 컬럼에 저장
submit = pd.read_csv('./sample_submission.csv')
submit['TARGET'] = pred

# 결과를 submit.csv 파일로 저장 (제출용)
submit.to_csv('submit.csv', index=False)


F1 Score on the validation set: 0.4312


## 전처리

미션 2 : 베이스라인 코드에서 스케일러를 추가해보세요.

(MinMax, MaxAbs, Standard, Robust 중 택 1)


In [7]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

x_train = train.drop(['ID', 'TARGET'], axis=1)
y_train = train['TARGET']

x_test = test.drop('ID', axis=1)

# 범주형 피처에 라벨 인코딩 적용
ordinal_features = ['요일', '범죄발생지']
for feature in ordinal_features:
    le = LabelEncoder()
    le.fit(x_train[feature])
    x_train[feature] = le.transform(x_train[feature])
    
    # 테스트 데이터에도 동일한 인코딩을 적용
    for label in np.unique(x_test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    x_test[feature] = le.transform(x_test[feature])

# 데이터를 학습용과 검증용으로 분할
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 스케일링 추가 (StandardScaler 사용)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

# 모델 학습
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

# 검증 데이터에 대한 예측
y_pred = model.predict(x_val)

# F1 스코어 계산
f1 = f1_score(y_val, y_pred, average='macro')
print(f'F1 Score on the validation set: {f1:.4f}')

# 테스트 데이터에 대한 예측 및 제출 파일 생성
pred = model.predict(x_test)
submit = pd.read_csv('./sample_submission.csv')
submit['TARGET'] = pred
submit.to_csv('submit.csv', index=False)


F1 Score on the validation set: 0.4303


미션 3 : 위 코드에서 Kfold를 활용해서 교차검증하는 코드로 변경해보세요.

미션 4 : Target값이 불균형하니 Stratified Kfold로 변경해보세요.

미션 5 : 데이터가 불균형하네요. 언더샘플링 기법 중 하나를 선택해서 사용해보세요.

미션 6 : 이번엔 오버 샘플링 기법 중 하나를 선택해 차이를 비교해보세요.

## 모델변경

미션 7 : 전처리가 된 코드에서 GradientBoostingClassifier로 모델을 변경해보세요.

미션 8 : 전처리가 된 코드에서 Catboost로 코드를 변경해보세요.

## 성능개선 : 하이퍼 파라미터 튜닝

미션 9: 그동안 가장 성능이 좋았던 모델을 Grid Search를 통해서 하이퍼파라미터들을 튜닝해보고 최적의 파라미터들을 출력해보세요(grid_search.best_params_)

## 성능개선 : 앙상블, 배깅

미션 10 : Grid Search를 통해 도출된 최적의 모델에 앙상블 기법 중 배깅을 활용해서 성능을 향상시켜보세요.

# AutoML ; PyCaret

PyCaret 공식 Document 중 QuickStart에 들어가시면 상세하게 잘 설명되어있습니다.

범죄유형 분류 데이터를 넣어서 PyCaret을 사용해보세요

In [8]:
# 설정 ; setup

In [9]:
# 모델간 비교


In [10]:
# 모델 분석

In [11]:
# 예측

In [12]:
# 모델 저장

미션 11 : PyCaret이 알려준 최고의 모델에 전처리 및 Grid Search를 적용해 하이퍼 파라미터를 튜닝해서 제출해보세요. 

## B코스 회원 전용

In [13]:
# GBM, XGBoost, LightGBM, Catboost 의 성능과 소요시간에 대한 표를 그리세요(pandas 활용)
# 예시)
#            GBM, XGBoost, LightGBM, Catboost
# f1_score    45    46        47        48
# 소요시간     30    20        15        10

In [14]:
# 데이터 불균형문제를 해소하기 위한 다양한 기법들이 있습니다.
# 그중 가장 좋은 방법은 무엇인지 실험결과로 입증해보세요.

In [15]:
# 다양한 출처에서 아이디어를 얻어와서 대회에 적용해보세요
# 출처는 세미나시간에 공유하거나 제출할겁니다. 출처 사이트도 같이 저장해두세요.